# Introduction
 In this assignment, you will do exercises on arcpy.Exists(), arcpy.Walk(), arcpy.da.Describe(), python list comprehension, arcpy.da.SearchCursor, and arcpy.da.InsertCursor(). In each section, a code block is provided but not ready to run. You need to add this notebook to ArcGIS Pro, modify each block to make it runable, keep the output message , and write an explanation of the code block. 

 Data preparation: 
 Use the same zip file from assignment 4 for this exercise. The zip file contains a geodatabase and a folder with shapefles. Download the data from the [data](../data) folder of the github website. Extract the zip file and use the geodatabase and the folder containing the shapefiles accordingly.  

## First, let's use ArcToolbox tools to convert the csv to a feature class (10 pnts)

- In ArcGIS Pro, search for "XYTable to Point"
- In the dialog, put the "point.csv" from this folder as the input table
- Name the output something like "point_fromxy"
- Before, you run the model, in the Run button, click the small triangle and select "Copy Python Command"
- Paste your python code below in the code block
- Run the code block to make sure it works
- Compare the python code and the ArcGIS Pro Geoprocessing pane dialog interface 


In [1]:
# To get the 10 points 
# Paste the python code from the above instruction here
# make sure you start with import arcpy
import arcpy
arcpy.management.XYTableToPoint(
    in_table=r"C:\Users\crsab\OneDrive\Documents\Programming\geog4057_Sabin\Assignment6\points.csv",
    out_feature_class=r"C:\Users\crsab\OneDrive\Documents\ArcGIS\Projects\Geog4057\Geog4057.gdb\points_fromXY",
    x_field="X",
    y_field="Y",
    z_field=None,
    coordinate_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision'
)

<Result 'C:\\Users\\crsab\\OneDrive\\Documents\\ArcGIS\\Projects\\Geog4057\\Geog4057.gdb\\points_fromXY'>

## Commentignt the example code:(20 pnts)
 - Using InsertCursor to do the same thing but project the geometry to a different spatial reference 

- The above code only imports the csv to a feature class but not converting from a GCS to a project coordinate system
- We will use the following code to do the work and project the data to "NAD 1983 USGS Contiguous USA Albers" system with the WKID = 102039
- Look the code up from here: https://epsg.io/102039

In [4]:
import arcpy
import csv
import os

# Edit the path names for your case
csv_file_path = r"C:\Users\crsab\OneDrive\Documents\Programming\geog4057_Sabin\Assignment6\points.csv"
geodatabase_path = r"C:\Users\crsab\OneDrive\Documents\ArcGIS\Projects\Geog4057\Geog4057.gdb\points_fromXY"
feature_class_name = "point_albers"

# Determine the number of fields and field names from the CSV header
with open(csv_file_path, 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    header = next(csv_reader)
    num_fields = len(header)

# Create SpatialReference objects: 
sr_gcs = arcpy.SpatialReference(4269) 
sr_albsers = arcpy.SpatialReference(102039)
#delete if the feature class exists

fcName = os.path.join(geodatabase_path,feature_class_name)

if arcpy.Exists(fcName):
    arcpy.management.Delete(fcName)


# Create a new point feature class

arcpy.CreateFeatureclass_management(geodatabase_path, feature_class_name, "POINT", spatial_reference=sr_albsers)
# Add fields to the feature class based on the CSV header

# first, read the exisint fields 
describe = arcpy.da.Describe(fcName)
field_names = [field.name for field in describe["fields"]]


for field in header:
    if field not in field_names:
        arcpy.AddField_management(fcName, field, "TEXT")

# Create a cursor for inserting point features with additional fields
cursor_fields = ["SHAPE@"] + header
with arcpy.da.InsertCursor(fcName, cursor_fields) as cursor:
    # Read data from the CSV file and create point features with attributes
    with open(csv_file_path, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        
        # Skip the header row (if present)
        next(csv_reader, None)
        
        for table_value in csv_reader:
            x = float(table_value[0]) # note that the first two fields in the csv must be longitude and latitude
            y = float(table_value[1])
            pnt = arcpy.PointGeometry(arcpy.Point(x,y),sr_gcs)
            pnt_alberse = pnt.projectAs(sr_albsers)

            row = (pnt_alberse,) + tuple(table_value)
            cursor.insertRow(row)

print(f"New point feature class '{feature_class_name}' created in '{geodatabase_path}' with {num_fields} fields.")



ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Feature Class Location: Dataset C:\Users\crsab\OneDrive\Documents\ArcGIS\Projects\Geog4057\Geog4057.gdb\points_fromXY does not exist or is not supported
Failed to execute (CreateFeatureclass).


**Edit this block to answer these three questions**

Q1: Line by line, explain what was done in the code. (10 pnts)
1. Lines (6-8)Establish paths for Data, GIS project geodatabase, and establish a feature class name
2. Lines (11-14)Read the CSV file so that the user understands the number of fields
3. Lines (17-18) Establish the spatial reference for the project 
4. Lines (21-24) adopt the spatial reference of the feature class
5. Line (29) create a new feature class (point)
6. Lines(33-34) List the fields that are in the attribute table of the feature class
7. Lines(37-39) Add a field to the feature class named "TEXT"
8. Lines(42-46) insert cursor to add a feature with attributes to feature class
9. Lines(49) skip row if header or null data
10. Lines(51-60)select features by attributes and print 

Q2: What do the cod 4269 and 102309 represent? (5 pnts)
The cod 4269 represents the NAD83 GCS, where the 102039 is a NAD 1983 USGS Contiguous USA projection

Q3: Explain what is SHAPE@ (5 pnts)
A token that is used to access a features geometry and attributes

## Use arcpy.Exist() (10 pnts)

- This code checks the existence of a specified dataset within an ArcGIS workspace.
- Fill the dataset_name and workspace_path variables with the database and the feature class name (points) from the last block
- Print a message indicating whether the dataset exists or not.

In [5]:
import arcpy
import os

# Prompt for user input
dataset_name = "points"
workspace_path = "Enter the workspace path: "
fullname = os.path.join(workspace_path,dataset_name)
# Check if the dataset exists
if arcpy.Exists(fullname):
    print(f"The dataset '{dataset_name}' exists in the workspace.")
else:
    print(f"The dataset '{dataset_name}' does not exist in the workspace.")

The dataset 'points' does not exist in the workspace.


**Edit this block to answer the question**

Q4: Line by line describe what was done by the code in the block above. (10 pnts) 

1.Lines(5-7)Establishing the workspace and dataset path.

2.Lines(9-12)the if-else statement finds if there is an existing dataset in the workspace path 

## Use arcpy.Walk() (10 pnts)

- This code uses arcpy.Walk() to iterate through all feature datasets in the geodatabase you just used.
- Modify the name of "workspace" to make the code work
- Run the code to list all the feature classes within each dataset.


In [6]:
import arcpy

# Define the workspace
workspace = "C:\Users\crsab\OneDrive\Documents\ArcGIS\Projects\Geog4057\Geog4057.gdb" # note that you need to put a full path name for this workspace, even running it in ArcGIS Pro. 

# Use arcpy.Walk() to iterate through feature datasets
for dirpath, dirnames, filenames in arcpy.da.Walk(workspace, datatype="FeatureClass"):
    for filename in filenames:
        print(f"Feature Class in {dirpath}: {filename}")

**Edit this block to answer the question**

Q5: line by line, describe what was done by the code in the block above. (10 pnts)

1. Line(4) defines the location of your geodatabase for the project
2. Line(7-9)Looks iinto the subfolder of the geodatabase folder to different datasets finding the desired feature or dataset. 





## Use List Comprehension (10 pnts)

- The following block uses list comprehension to generate a list of .shp files in a specified folder.
- Choose the folder name from assignment 4 data/paris subfolder where many shapefiles are located and use it for folder_path
- Run the code to print the list of file names


- In the second block, the code combines the os.walk() function with the list comprehension to  list all shapefiles in a folder including subfolders



In [8]:
import os

# Specify the folder path
folder_path = r"C:\Users\crsab\OneDrive\Documents\Programming\geog4057_Sabin\ModelBuilder_data\ParisData\ParisTransportation\Metro"

# Use list comprehension to generate a list of .shp files
shp_files = [file for file in os.listdir(folder_path) if file.endswith(".shp")]

# Print the list of .shp files
print("Shapefiles in the folder:")
for shp_file in shp_files:
    print(shp_file)

Shapefiles in the folder:
Metro_Lines.shp
Metro_LinesAOI.shp
Metro_Stations.shp


In [9]:
import os
# Specify the folder path
folder_path = r"C:\Users\crsab\OneDrive\Documents\Programming\geog4057_Sabin\ModelBuilder_data\ParisData\ParisTransportation\Metro"

print("Shapefiles in the folder:")
for root, dirs, files in os.walk(folder_path):
    shp_files = [file for file in files if file.endswith(".shp")]
    for shp_file in shp_files:
            print(shp_file)

Shapefiles in the folder:
Metro_Lines.shp
Metro_LinesAOI.shp
Metro_Stations.shp


**Edit this block to answer the question**

Q6: Describe what was done by the code in first blocks above. (5 pnts)

this code made a list of all files ending in .shp

Q7: Describe what was done by the code in second blocks above. (5 pnts)

this code explored subfolders to return any files ending .shp

## Use arcpy.da.SearchCursor (10 pnts)

- This code uses arcpy.da.SearchCursor to extract attribute information from a feature class.
- Enter the name of the "points" feature class and the field(s) from the feature class to extract.
- Run and display the extracted data.


In [10]:
import arcpy

# Prompt for user input
fc_path = r"C:\Users\crsab\OneDrive\Documents\ArcGIS\Projects\Geog4057\Geog4057.gdb\points_fromXY"
fields_to_extract = ["OBJECTID", "X", "Y", "Field1", "Field2", "Field3"]

# Use arcpy.da.SearchCursor to extract data
with arcpy.da.SearchCursor(fc_path, fields_to_extract) as cursor:
    print("Extracted Data:")
    for row in cursor:
        print([row[i] for i in range(len(fields_to_extract))])

Extracted Data:
[1, -118.2437, 34.0522, 10, 15, 20]
[2, -73.935242, 40.73061, 8, 11, 14]
[3, -95.369804, 29.760427, 12, 18, 22]
[4, -121.886329, 37.338208, 6, 9, 12]
[5, -87.629798, 41.878113, 15, 22, 28]
[6, -97.743061, 30.267153, 11, 16, 21]
[7, -122.419416, 37.774929, 9, 13, 17]
[8, -118.242766, 34.052235, 14, 21, 26]
[9, -71.05888, 42.360081, 7, 10, 13]
[10, -81.69436, 41.499321, 12, 17, 22]


**Edit this block to answer the question**

Q8: Line by line, explain what was done in the code. 10 pnts 

1. Lines(4-5) establishing path to feature class and establishing which fields to extract

2. Lines(8-11) search cursor finds records of data for the determined fields and print returns those records based on the user determined order

## Use addField and field Calculator (20 pnts)

- Run the following code block (with "points" feature class added to the last map before you switched into the notebook interface)

In [12]:
import arcpy
fc = r"C:\Users\crsab\OneDrive\Documents\ArcGIS\Projects\Geog4057\Geog4057.gdb\points_fromXY"
newfieldName = "all"
arcpy.AddField_management(fc, newfieldName, "DOUBLE")
expression = "sum(!Field1!,!Field2!,!Field3!)"
codeblock = """
def sum(*fields):
    sum = 0
    for field in fields:
        sum += field
    return sum
"""
arcpy.CalculateField_management(fc, newfieldName, expression, "", codeblock)

<Result 'C:\\Users\\crsab\\OneDrive\\Documents\\ArcGIS\\Projects\\Geog4057\\Geog4057.gdb\\points_fromXY'>

**Edit this block to answer the questions**

Q9: line by line, describe what was done by the code in the block above. 10 pnts

1. Lines(2) establish path to feature class

2. Lines(3) define function newfieldname

3. Lines(4) adding a new field to attribute table called "all" with parameter format "double"

4. Lines(5) calculating sum of Field1 + Field2 + Field3 

5. Lines(6-13) returning the value of the calculate to the new field "all" column

Q10: Open the attribute table of "points" and check if the attribute table has a new filed "all" and correct values. Right-click the "all" field and click field calculator. Describe what you see in the field calculator interface. Compare the python code versus the field calculator interface. 10 pnts

    1. The attribute table did add the new field "all" and the values are correct.  For example the value in row 1 for "all" is 45(which is the total sum of (Field1=10)+(Field2=15)+(Field3=20).  Right click on the "all" field to open the calculate field interface and you will find the tool to input commands.  Expression is "all" = with an empty dialog box.  User can double click the other fields and separate them with "+" to create an expression that yields the same result as code.
         !Field1! + !Field2! + !Field3!

## Submit the assignment ontime (10 pnts)

- Late penalty is 10 points from here